# 1. Update Download File lên Download cả File và Folder bằng cách cập nhật API

# 2. IPA Paralles 

Chạy song song việc generate_story và generate_ipa, ... được ko nhỉ @routes.py 

Nghĩ cách thiết kế chạy song song của @scripts.py 

và @ContentGenerator.js 

Tôi sẽ giải thích cách triển khai chạy song song các scripts:

### 1. Phía Backend (Python/Flask):

```python:backend/api/files/routes.py
@bp.route('/run-parallel', methods=['POST'])
def run_parallel_scripts():
    # ... code như trên ...
```

**Giải thích các bước chính:**

1. **Nhận danh sách scripts:**
   - API nhận một mảng tên scripts cần chạy từ frontend
   - Kiểm tra tính hợp lệ của danh sách

2. **Xử lý song song:**
   - Sử dụng `ThreadPoolExecutor` để tạo pool các threads
   - Mỗi script sẽ chạy trên một thread riêng biệt
   - Không block main thread của Flask

3. **Thu thập kết quả:**
   - Sử dụng `concurrent.futures.as_completed()` để lấy kết quả ngay khi script hoàn thành
   - Không cần đợi tất cả scripts chạy xong mới trả kết quả
   - Lưu trạng thái của từng script (completed/error)

### 2. Phía Frontend (React):

```javascript:frontend/src/features/ContentGenerator/ContentGenerator.js
function ContentGenerator() {
    // ... code như trên ...
}
```

**Giải thích các bước chính:**

1. **Quản lý state:**
   ```javascript
   const [selectedScripts, setSelectedScripts] = useState([]); // Danh sách scripts được chọn
   const [scriptStatus, setScriptStatus] = useState({}); // Trạng thái của từng script
   const [loading, setLoading] = useState(false); // Trạng thái đang chạy
   ```

2. **Chọn scripts:**
   - Người dùng có thể chọn nhiều scripts cùng lúc
   - Mỗi script được chọn sẽ được highlight
   - Có thể bỏ chọn script bằng cách click lại

3. **Chạy song song:**
   ```javascript
   const runParallelScripts = async () => {
       // Gửi request tới API với danh sách scripts đã chọn
       // Cập nhật trạng thái real-time cho từng script
   }
   ```

4. **Hiển thị trạng thái:**
   - Mỗi script có visual feedback riêng
   - Showing loading indicator khi đang chạy
   - Hiển thị kết quả hoặc lỗi cho từng script

### 3. Luồng xử lý:

1. **Người dùng:**
   - Chọn các scripts muốn chạy
   - Click nút "Run Selected Scripts"

2. **Frontend:**
   - Gửi request với danh sách scripts
   - Hiển thị trạng thái "running" cho các scripts được chọn

3. **Backend:**
   - Nhận request và tạo thread pool
   - Chạy các scripts song song
   - Gửi kết quả về từng script

4. **Frontend nhận kết quả:**
   - Cập nhật UI với trạng thái mới
   - Hiển thị kết quả hoặc lỗi
   - Reset trạng thái để sẵn sàng chạy lần tiếp

### 4. Ưu điểm của cách làm này:

1. **Hiệu quả:**
   - Chạy nhiều scripts cùng lúc
   - Không block main thread
   - Tận dụng được nhiều core CPU

2. **User Experience:**
   - UI thân thiện, dễ sử dụng
   - Feedback realtime
   - Có thể thấy tiến độ từng script

3. **Bảo trì:**
   - Code có cấu trúc rõ ràng
   - Dễ thêm scripts mới
   - Dễ debug khi có lỗi

4. **Mở rộng:**
   - Dễ dàng thêm tính năng mới
   - Có thể thêm progress bar
   - Có thể thêm cancel function

I'll help you modify both files to support parallel script execution. Here are the changes needed:

1. First, let's modify the Flask backend to handle parallel script execution:

```python:backend/api/files/routes.py
from flask import Blueprint, request, jsonify, send_file
import asyncio
import concurrent.futures
from functools import partial

# ... existing imports and setup ...

# Add new route for parallel script execution
@bp.route('/run-parallel', methods=['POST'])
def run_parallel_scripts():
    try:
        scripts = request.json.get('scripts', [])
        if not scripts:
            return jsonify({'error': 'No scripts provided'}), 400

        # Create thread pool for parallel execution
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Map scripts to futures
            future_to_script = {
                executor.submit(run_single_script, script): script 
                for script in scripts
            }
            
            results = {}
            # Collect results as they complete
            for future in concurrent.futures.as_completed(future_to_script):
                script_name = future_to_script[future]
                try:
                    results[script_name] = {
                        'status': 'completed',
                        'result': future.result()
                    }
                except Exception as e:
                    results[script_name] = {
                        'status': 'error',
                        'error': str(e)
                    }

        return jsonify({
            'success': True,
            'results': results
        })

    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

def run_single_script(script_name):
    # Implement your actual script execution logic here
    # This is just a placeholder
    return f"Executed {script_name}"
```

2. Then, let's update the React component to handle parallel execution:

```javascript:frontend/src/features/ContentGenerator/ContentGenerator.js
function ContentGenerator() {
    const [loading, setLoading] = useState(false);
    const [error, setError] = useState(null);
    const [scriptStatus, setScriptStatus] = useState({});
    const [selectedScripts, setSelectedScripts] = useState([]);

    // Available scripts
    const availableScripts = [
        'generate_story',
        'generate_ipa',
        'generate_meaning',
        'gen_answer_pic'
    ];

    // Handle script selection
    const toggleScriptSelection = (scriptName) => {
        setSelectedScripts(prev => 
            prev.includes(scriptName)
                ? prev.filter(s => s !== scriptName)
                : [...prev, scriptName]
        );
    };

    // Run multiple scripts in parallel
    const runParallelScripts = async () => {
        if (selectedScripts.length === 0) {
            setError('Please select at least one script');
            return;
        }

        try {
            setLoading(true);
            setError(null);
            
            // Set all selected scripts to 'running' status
            const newStatus = {};
            selectedScripts.forEach(script => {
                newStatus[script] = 'running';
            });
            setScriptStatus(newStatus);

            // Call the parallel execution API
            const response = await runScriptApi({
                method: 'POST',
                endpoint: '/run-parallel',
                data: { scripts: selectedScripts }
            });

            // Update status based on results
            const finalStatus = {};
            Object.entries(response.results).forEach(([script, result]) => {
                finalStatus[script] = result.status;
            });
            setScriptStatus(finalStatus);

        } catch (error) {
            console.error('Parallel execution error:', error);
            setError(`Error running scripts: ${error.message}`);
            
            // Set all selected scripts to error status
            const errorStatus = {};
            selectedScripts.forEach(script => {
                errorStatus[script] = 'error';
            });
            setScriptStatus(errorStatus);
        } finally {
            setLoading(false);
        }
    };

    return (
        <div className="generator-container">
            <h3>Generate Content</h3>
            <div className="script-buttons">
                {availableScripts.map(script => (
                    <button
                        key={script}
                        onClick={() => toggleScriptSelection(script)}
                        className={`script-button ${scriptStatus[script] || ''} 
                            ${selectedScripts.includes(script) ? 'selected' : ''}`}
                        disabled={loading}
                    >
                        {script.replace(/_/g, ' ').toUpperCase()}
                        {scriptStatus[script] === 'running' && 
                            <span className="loading">...</span>}
                    </button>
                ))}
            </div>
            
            <button 
                className="run-parallel-button"
                onClick={runParallelScripts}
                disabled={loading || selectedScripts.length === 0}
            >
                Run Selected Scripts
            </button>
            
            {error && <div className="error-message">{error}</div>}
        </div>
    );
}
```

Key changes and features:

1. **Backend Changes**:
- Added new `/run-parallel` endpoint that accepts multiple scripts
- Uses ThreadPoolExecutor for parallel execution
- Returns results as they complete
- Maintains individual status for each script

2. **Frontend Changes**:
- Added script selection functionality
- New "Run Selected Scripts" button
- Shows status for each script individually
- Improved error handling
- Visual feedback for selected scripts

3. **Additional CSS needed**:
```css
.script-button.selected {
    border: 2px solid #4CAF50;
}

.run-parallel-button {
    margin-top: 20px;
    padding: 10px 20px;
    background-color: #4CAF50;
    color: white;
    border: none;
    border-radius: 4px;
    cursor: pointer;
}

.run-parallel-button:disabled {
    background-color: #cccccc;
    cursor: not-allowed;
}
```

This implementation allows users to:
1. Select multiple scripts
2. Run them in parallel
3. See individual status for each script
4. Get feedback on completion or errors
5. Cancel selection and start over

The backend handles the parallel execution efficiently using thread pools, while the frontend provides a user-friendly interface for managing multiple script executions.

# 3. Copy cách thức Dựa vào các file này, viết file @ShowExampleData.js 
---------
Để  show data trong mục example hiển thị lên UI, và có thể download xuống 

TUYỆT ĐỐI KHÔNG SỬA LÀM ẢNH HƯỞNG ĐẾN CÁC PHẦN KHÁC NHÉ 




Cho example data nhỏ lại đi, vào 1 góc của khung của @FileUpload.css @FileUpload.js 
